In [1]:
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras


In [3]:
df=pd.read_csv("train.csv",header=None)

In [4]:
df.head()

,0,1,2
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...
1,5,Inspiring,I hope a lot of people hear this cd. We need m...
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
4,5,Too good to be true,Probably the greatest soundtrack in history! U...


In [5]:
df.columns=['Rating','ReviewTitle','Review']

In [6]:
df1=df[df.Rating==1].head(2000)
df2=df[df.Rating==2].head(2000)
df3=df[df.Rating==3].head(2000)
df4=df[df.Rating==4].head(2000)
df5=df[df.Rating==5].head(2000)

In [7]:
df=df1.append([df2,df3,df4,df5])

In [8]:
df.Rating.value_counts()

5    2000
4    2000
3    2000
2    2000
1    2000
Name: Rating, dtype: int64

In [9]:
df=df.reset_index(drop=True)

In [10]:
def markings(x):
    if x==5:
        y=0
    else:
        y=1
    return y 

In [11]:
#df.Rating=df.Rating.apply(lambda x:markings(x))

In [12]:
df.Rating.value_counts()

5    2000
4    2000
3    2000
2    2000
1    2000
Name: Rating, dtype: int64

In [13]:
df.to_csv("traindata.csv",index=False)

In [2]:
df=pd.read_csv("traindata.csv")

In [3]:
import nltk


from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))

In [4]:
def remove_stopwords(data):
  data['review without stopwords'] = data['Review'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result

In [5]:
df['Review'] = df['Review'].str.lower()


In [6]:
data_without_stopwords = remove_stopwords(df)


In [7]:
data_without_stopwords.head()

,Rating,ReviewTitle,Review,review without stopwords
0,1,Buyer beware,"this is a self-published book, and if you want...","self-published book, want know why--read parag..."
1,1,The Worst!,a complete waste of time. typographical errors...,"complete waste time. typographical errors, poo..."
2,1,Oh please,i guess you have to be a romance novel lover f...,"guess romance novel lover one, discerning one...."
3,1,Awful beyond belief!,i feel i have to write to keep others from was...,feel write keep others wasting money. book see...
4,1,Another Abysmal Digital Copy,"rather than scratches and insect droppings, th...","rather scratches insect droppings, one random ..."


In [8]:
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')


In [9]:

reviews_list = []
for i in range(len(data_without_stopwords.clean_review)):
    reviews_list.append(data_without_stopwords.clean_review[i])
 
sentiment = data_without_stopwords['Rating']

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, sentiment, test_size=0.3, random_state = 45)


In [18]:
len(X_train)

7000

In [19]:
Y_test.value_counts()

3    627
1    614
2    600
4    593
5    566
Name: Rating, dtype: int64

In [20]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

words_to_index = tokenizer.word_index

In [21]:
def read_glove_vector(glove_vec):
    with open(glove_vec, 'r', encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            w_line = line.split()
            curr_word = w_line[0]
            word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
    return word_to_vec_map

In [22]:
word_to_vec_map = read_glove_vector('/Users/shinjini/Downloads/glove/glove.6B.50d.txt')


In [23]:
maxLen = 150


In [24]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['tuition'].shape[0]



In [25]:
vocab_len
embed_vector_len

50

In [26]:
len(words_to_index)

25890

In [27]:
emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        emb_matrix[i] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)

In [28]:
def amazon_rating(input_shape):
    X_indices = Input(input_shape)
    embeddings = embedding_layer(X_indices)
    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.1)(X)
    X = LSTM(128, return_sequences=True)(X)
    X = Dropout(0.1)(X)
    X = LSTM(128)(X)
    X = Dense(5, activation='softmax')(X)
    model = Model(inputs=X_indices, outputs=X)
    return model



In [29]:
model = amazon_rating((maxLen,))
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 50)           1294500   
_________________________________________________________________
lstm (LSTM)                  (None, 150, 128)          91648     
_________________________________________________________________
dropout (Dropout)            (None, 150, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 150, 128)          131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584

In [30]:
X_train_indices = tokenizer.texts_to_sequences(X_train)
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

In [31]:
Y_train = pd.get_dummies(Y_train).values
Y_test=pd.get_dummies(Y_test).values

In [32]:
#Y.shape


In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_indices, Y_train, batch_size=128, epochs=15)

Epoch 1/15
55/55 [==============================] - 39s 646ms/step - loss: 1.6100 - accuracy: 0.1994
Epoch 2/15
55/55 [==============================] - 36s 656ms/step - loss: 1.6097 - accuracy: 0.1976
Epoch 3/15
55/55 [==============================] - 39s 706ms/step - loss: 1.6098 - accuracy: 0.1974
Epoch 4/15
55/55 [==============================] - 38s 686ms/step - loss: 1.6098 - accuracy: 0.1993
Epoch 5/15
55/55 [==============================] - 43s 780ms/step - loss: 1.6097 - accuracy: 0.2011
Epoch 6/15
55/55 [==============================] - 38s 692ms/step - loss: 1.6096 - accuracy: 0.2034
Epoch 7/15
55/55 [==============================] - 38s 700ms/step - loss: 1.6096 - accuracy: 0.2069
Epoch 8/15
55/55 [==============================] - 38s 700ms/step - loss: 1.6097 - accuracy: 0.1953
Epoch 9/15
55/55 [==============================] - 39s 709ms/step - loss: 1.6096 - accuracy: 0.2049
Epoch 10/15
55/55 [==============================] - 39s 713ms/step - loss: 1.6095 - accura

In [38]:
#testdata=pd.read_csv("test.csv")

In [39]:
#df.head()


,Rating,ReviewTitle,Review,review without stopwords,clean_review
0,1,Buyer beware,"this is a self-published book, and if you want...","self-published book, want know why--read parag...",self published book want know why read parag...
1,1,The Worst!,a complete waste of time. typographical errors...,"complete waste time. typographical errors, poo...",complete waste time typographical errors poo...
2,1,Oh please,i guess you have to be a romance novel lover f...,"guess romance novel lover one, discerning one....",guess romance novel lover one discerning one ...
3,1,Awful beyond belief!,i feel i have to write to keep others from was...,feel write keep others wasting money. book see...,feel write keep others wasting money book see...
4,1,Another Abysmal Digital Copy,"rather than scratches and insect droppings, th...","rather scratches insect droppings, one random ...",rather scratches insect droppings one random ...


In [ ]:
test